## AoI Identification from screenshots
Status: Implemented
 -- Fully automatized

In [1]:
import json
from pathlib import Path


from tqdm.notebook import tqdm
from utils.utils import *

In [2]:
AOI_SUMMARY_PATH.parent.mkdir(parents=True, exist_ok=True)

### AOI detection & retrieval from (screenshot) images

In [3]:
screenshots = SCREENSHOTS_PATH_AOI.iterdir()

no_aois_found = []
all_aois = {}
for screenshot_path in tqdm(screenshots):
    snippet = screenshot_path.stem
    # if not snippet in redone:
    #     continue
    print(snippet)
    image, contours = detect_possible_AOI_contours_from_screenshot(
        screenshot_path)
    if len(contours) == 0:
        print(f'no contours found for {snippet}')
    rectangles = get_rectangles_from_contours(contours)
    print([r[1] for r in rectangles])
    aois = get_AOIs_from_rectangles(screenshot_path, image, rectangles)
    if len(aois) == 0:
        print(f'no aois found for {snippet}')
        no_aois_found.append(snippet)
    # with open(AOI_PATH/f'{snippet}.json', 'w') as f:
    #     json.dump(aois, f, indent=4, sort_keys=True)
    all_aois[snippet] = aois  # store with snippet as key

0it [00:00, ?it/s]

-clean-v0
[{'top': 369, 'left': 555, 'bottom': 422, 'right': 981}, {'top': 373, 'left': 739, 'bottom': 417, 'right': 976}, {'top': 377, 'left': 743, 'bottom': 413, 'right': 794}]
-clean-v1
[{'top': 369, 'left': 555, 'bottom': 422, 'right': 981}, {'top': 373, 'left': 739, 'bottom': 417, 'right': 976}, {'top': 377, 'left': 743, 'bottom': 413, 'right': 794}]
-clean-v2
[{'top': 369, 'left': 555, 'bottom': 422, 'right': 981}, {'top': 373, 'left': 739, 'bottom': 417, 'right': 976}, {'top': 377, 'left': 743, 'bottom': 413, 'right': 794}]
-obf-v0
[{'top': 369, 'left': 555, 'bottom': 422, 'right': 1071}, {'top': 373, 'left': 739, 'bottom': 417, 'right': 1066}, {'top': 377, 'left': 781, 'bottom': 413, 'right': 845}]
-obf-v1
[{'top': 369, 'left': 555, 'bottom': 422, 'right': 1083}, {'top': 373, 'left': 739, 'bottom': 417, 'right': 1078}, {'top': 377, 'left': 781, 'bottom': 413, 'right': 845}]
-obf-v2
[{'top': 369, 'left': 555, 'bottom': 422, 'right': 1083}, {'top': 373, 'left': 739, 'bottom': 417

In [4]:
# manual adaptions for data correction, catching those aois that couldn't be recognized
all_aois['5-clean-v1'][AOI_WEXP] = {POS_TOP: 270,
                                    POS_LEFT: 740, POS_BOTTOM: 311, POS_RIGHT: 1358}
all_aois['5-obf-v1'][AOI_WEXP] = {POS_TOP: 270,
                                  POS_LEFT: 740, POS_BOTTOM: 311, POS_RIGHT: 1346}
for s in ['49-clean-v0', '49-clean-v1', '49-clean-v2']:
    all_aois[s][AOI_BLOCK] = {pos: v+{POS_LEFT: 0, POS_RIGHT: AOI_PADDING, POS_BOTTOM: AOI_PADDING,
                                      POS_TOP: -AOI_PADDING}[pos] for pos, v in all_aois[s][AOI_STAT].items()}

In [5]:
with open(AOI_SUMMARY_PATH, 'w') as f:
    json.dump(all_aois, f, indent=4, sort_keys=True)

In [6]:
no_aois_found

[]

In [7]:
with open(AOI_SUMMARY_PATH) as f:
    all_aois = json.load(f)
# sanity check 1: compare results in all_aois to markings in original code snippets
for snippet, snippet_aois in tqdm(all_aois.items()):
    code_snippet_path = get_code_snippet_path(snippet, aoi_version=True)
    with open(code_snippet_path) as f:
        code_snippet = f.read()
    count_aois = code_snippet.count('aoi=')
    code_aois = re.findall(CODE_AOI_PATTERN, code_snippet)
    assert count_aois == len(
        code_aois), f'amount of AoIs counted {count_aois} should be identical to amount of AoIs identified {len(code_aois)}'
    if len(snippet_aois) != count_aois:
        print(f'{snippet}: Amount of AoIs identified in code and in screenshot differ: {code_aois} vs. {snippet_aois.keys()} from image')
    elif set(snippet_aois.keys()) != set(code_aois):
        print(f'{snippet}: AoIs identified in code and in screenshot differ: {code_aois} in code vs. {snippet_aois.keys()} from image')

  0%|          | 0/144 [00:00<?, ?it/s]

In [8]:
# Sanity check 2: check whether all snippet pairs have equal AoIs within
unmatching_snippets = check_matching_AOIs(all_aois)
print(unmatching_snippets)

  0%|          | 0/144 [00:00<?, ?it/s]

[]
